In [1]:
#pip install findspark

In [2]:
import findspark


In [3]:
import os
os.getcwd()

'D:\\Kafka'

In [4]:
findspark.init()

In [5]:
import pyspark
import time

In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import pyspark.sql.types as TP
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
import sys
from pyspark.streaming.kafka import KafkaUtils

In [7]:
sc = SparkContext(appName="kafka_tweet1")

In [8]:
spark = SparkSession(sc)

In [9]:
my_schema = TP.StructType([
  TP.StructField(name= 'id',          dataType= TP.StringType(),  nullable= True),
  TP.StructField(name= 'label',       dataType= TP.IntegerType(),  nullable= True),
  TP.StructField(name= 'tweet',       dataType= TP.StringType(),   nullable= True)
])

In [10]:
my_data= spark.read.csv('D:/PySpark/PySpark-master/spark_streaming/datasets/twitter_sentiments.csv', schema= my_schema, header=True)

In [11]:
my_data.show(5)

+---+-----+--------------------+
| id|label|               tweet|
+---+-----+--------------------+
|  1|    0| @user when a fat...|
|  2|    0|@user @user thank...|
|  3|    0|  bihday your maj...|
|  4|    0|#model   i love u...|
|  5|    0| factsguide: soci...|
+---+-----+--------------------+
only showing top 5 rows



In [12]:
stage_1 = RegexTokenizer(inputCol= 'tweet' , outputCol= 'tokens', pattern= '\\W')

stage_2 = StopWordsRemover(inputCol= 'tokens', outputCol= 'filtered_words')

stage_3 = Word2Vec(inputCol= 'filtered_words', outputCol= 'vector', vectorSize= 100)

model = LogisticRegression(featuresCol= 'vector', labelCol= 'label')

In [13]:
pipeline= Pipeline(stages=[stage_1, stage_2, stage_3, model])

In [14]:
pipelineFit = pipeline.fit(my_data)

In [15]:
def get_prediction(tweet_text):
    try:
    # filter the tweets whose length is greater than 0
        #tweet_text = tweet_text.filter(lambda x: len(x) > 0)
    # create a dataframe with column name 'tweet' and each row will contain the tweet
        #rowRdd = tweet_text.map(lambda w: Row(tweet=w))
    # create a spark dataframe
        #wordsDataFrame = spark.createDataFrame(rowRdd)
    # transform the data using the pipeline and get the predicted sentiment
        pipelineFit.transform(wordsDataFrame).select('tweet','prediction').show()
    except : 
        print('No data')

In [16]:
ssc = StreamingContext( sc, batchDuration= 3)
        

In [17]:
def readMyRddsFromKafkaStream( readRdd ):
  # Put RDD into a Dataframe
    df = spark.read.json( readRdd )
    df.registerTempTable( "temporary_table" )
    df = spark.sql( """SELECT tweet FROM temporary_table """ )
    pipelineFit.transform(df).select('tweet','prediction').show()

In [18]:
ssc = StreamingContext(sc,3)
kvs= KafkaUtils.createDirectStream(ssc, topics=['tweet1'],kafkaParams={'metadata.broker.list':'localhost:9092'})



data = kvs.map( lambda tuple: tuple[1] )
message= data.foreachRDD( lambda yourRdd: readMyRddsFromKafkaStream( yourRdd ) )



    
ssc.start()
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o214.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\BigData\SPARK_2.4\python\pyspark\sql\utils.py", line 63, in deco
    return f(*a, **kw)
  File "C:\BigData\SPARK_2.4\python\lib\py4j-0.10.7-src.zip\py4j\protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling o21.sql.
: org.apache.spark.sql.AnalysisException: cannot resolve '`tweet`' given input columns: []; line 1 pos 7;
'Project ['tweet]
+- SubqueryAlias `temporary_table`
   +- LogicalRDD false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$ano

+--------------------+----------+
|               tweet|prediction|
+--------------------+----------+
|International rep...|       0.0|
+--------------------+----------+

+--------------------+----------+
|               tweet|prediction|
+--------------------+----------+
|Should page thous...|       0.0|
+--------------------+----------+

+--------------------+----------+
|               tweet|prediction|
+--------------------+----------+
|Establish recogni...|       0.0|
+--------------------+----------+

+--------------------+----------+
|               tweet|prediction|
+--------------------+----------+
|Very she mouth he...|       0.0|
+--------------------+----------+

+--------------------+----------+
|               tweet|prediction|
+--------------------+----------+
|Necessary probabl...|       0.0|
+--------------------+----------+

+--------------------+----------+
|               tweet|prediction|
+--------------------+----------+
|Building spend ma...|       0.0|
+--------